Author: Yibing Sun
Project: Visual Hate
Data: Tweets that incorporate hate slurs and have attachments
Share to: CAMER, SJMC, UW-Madison

I usually share this when I’m receiving requests about sharing scripts: 
    
The scripts that I have written are usually created for the particular project. 
If you are going to use the scripts for your other projects, please be respectful to the original creator (me or others). 
Ask for permission before you use it. You can either ask for interests in co-authoring or acknowledging their contribution in the publication stage. I do the same thing to acknowledge other people’s efforts as well. Thank you for your understanding! 
    
Yibing Sun

In [ ]:
import json
import pandas as pd
import os
import requests
import time

In [ ]:
def concatenate_json_files(filenames, output_filename):
    data = []
    for filename in filenames:
        if filename.endswith('json'):
            with open(filename, "r") as file:
                file_data = json.load(file)
                data.append(file_data)
    with open(output_filename, "w") as outfile:
        json.dump(data, outfile)

In [ ]:
def get_filenames_in_folder(folder_path):
    return [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]

In [ ]:
#get a list of subfolders for a specific month
file_path = "YOUR PATH"
folder_list = os.listdir("YOUR PATH")
subf_list = []
for i in range(0, len(folder_list)):
    subf_list.append(os.path.join(file_path, folder_list[i]))
subf_list

In [ ]:
# create a list of files within that day
for folder in subf_list:
    filenames = get_filenames_in_folder(folder)
#     filenames = filenames[:]
    print(filenames)
    day = filenames[0].split('_')[0]
#     print(filenames)
    filename_list = [folder+'/' + filename for filename in filenames]
#     print(filename_list)
    concatenate_json_files(filename_list, "YOUR OWN PATH WITH FORMAT".format(day)) 
    print(day)

In [ ]:
def get_filenames_in_folder(folder_path):
    return [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]

In [ ]:
test = get_filenames_in_folder('YOUR PATH')
filenames_json = []
for f in test:
    if f.endswith('json'):
        filenames_json.append(f)
filenames_json = ['PART OF YOUR FORMATTED PATH' + x for x in filenames_json]
filenames_json

In [ ]:
output = pd.DataFrame(columns = ['Date', 'Tweets_Volume', 'Retweet_Count','Reply_Count',
                                 'Like_count', 'Quote_count','Sensitive', 'Retweet_N', 'Retweet_De'],
                      index=range(YOUR CHOSEN NUMBER))
for i in range(len(filenames_json)):
    tweets_res = []
    for line in open(filenames_json[i], encoding = "ISO-8859-1"):
        try:
            tweets_res.append(json.loads(line))
        except ValueError as e:
            tweets_res.append("Error")
            print(e)
            print(line)
    tweet_flat = []
    for s in tweets_res:
        for j in s:
            if j.get('data'):
                for k in j['data']:
                    tweet_flat.append(k)
    df = pd.DataFrame(tweet_flat)
    df = df.sort_values("created_at")
    df = df.reindex(sorted(df.columns), axis=1)
    #create another column referenced id for future vlookup
    df['referenced_id'] = df['referenced_tweets']
    for i1 in range(len(df)):
        if type(df.iloc[i1, -1]) != float:
            df.iloc[i1, -1] = df.iloc[i1, -1][0]['id']

    ### Dataframe 2: retweet entities
    retweet_flat = []
    for tweet in tweets_res:
        for j1 in tweet:
            if j1.get('includes').get('tweets'):
                for k1 in j1['includes']['tweets']:
                    retweet_flat.append(k1)
    df1 = pd.DataFrame(retweet_flat)
    df1=df1.rename(columns = {'id':'referenced_id', 'text': 'referenced_text', 'attachments': 'referenced_attachment'})
    df1 = df1[['referenced_id','referenced_text', 'referenced_attachment']]
    Retweet_N = df1.shape[0]
    df1['referenced_attachment'] = df1['referenced_attachment'].astype(str)
    df1 = df1.drop_duplicates()
    Retweet_De = df1.shape[0]

    df3 = pd.merge(df, df1, on="referenced_id", how = "left")

    prc = pd.json_normalize(df3.public_metrics, sep=',')
    test = df3.join(prc)
    Tweets_Volume = test.shape[0]

    Retweet_Count = round(test["retweet_count"].mean(),2)
    Reply_Count = round(test["reply_count"].mean(),2)
    Like_count = round(test["like_count"].mean(), 2)
    Quote_count = round(test["quote_count"].mean(), 2)
    try:
        Sensitive = test['possibly_sensitive'].value_counts()[1]
    except:
        Sensitive = 0
    

    name = filenames_json[i].split('/')[-1].split('.')[0]
    test.to_csv('YOUR FORMATTED PATH'.format(name), index = False)
    print(name)
    
    output.iloc[i,0] = name
    output.iloc[i,1] = Tweets_Volume
    output.iloc[i,2] = Retweet_Count
    output.iloc[i,3] = Reply_Count
    output.iloc[i,4] = Like_count
    output.iloc[i,5] = Quote_count
    output.iloc[i,6] = Sensitive
    output.iloc[i,7] = Retweet_N
    output.iloc[i,8] = Retweet_De

In [ ]:
output

In [ ]:
output.to_csv('YOUR PATH/summary.csv') #Summarized csv